In [85]:
import json

In [141]:
with open('./src/metadata.json', 'r') as f:
    jtxt = f.read()

j = json.loads(jtxt)
def dig_dict(dictionary, pretab=0, indent=0):
    config_string = ''
    for key, val in dictionary.items():
        config_string += '\t'*(pretab + indent) + str(key) + '\n'
        print(key)
        print('\n\n')
        if isinstance(val, dict):
            dig_dict(val, indent+1)
        else:
            config_string += '\t'*(pretab + indent+1) + str(val) + '\n'
    return config_string

In [191]:
j.keys()
type_dict = {}
tag_dict = {}
param_dict = {}
optional_dict = {}

j['schema'] = {k: v for k,v in j['schema'].items() if 'region' not in k.lower()}
j['schema'] = {k: v for k,v in j['schema'].items() if 'inst' not in k.lower()}
for arche, xml in j['schema'].items():
    #print(xml)
    schema_dict = xmltodict.parse(xml)
    type_dict[arche] = {}
    tag_dict[arche] = {}
    optional_dict[arche] = []
    param_dict[arche] = {'oneormore': [],
                         'one': []}
    if 'interleave' not in schema_dict.keys():
        continue
    for op_el in schema_dict['interleave']:
        if op_el == 'element':
            optional = False
            if isinstance(schema_dict['interleave'][op_el], list):
                for param in schema_dict['interleave'][op_el]:
                    name = param['@name']
                    if 'interleave' in param:
                        print('%s Cant do man' %param)
                        continue
                    if 'data' in param:
                        param_dict[arche]['one'].append(name)
                        type_dict[arche][name] = param['data']['@type']
                        continue
                    if 'oneOrMore' in param:
                        param_dict[arche]['oneormore'].append(name)
                        tag_dict[arche][name] = param['oneOrMore']['element']['@name']
                        if 'interleave' in param['oneOrMore']['element'].keys():
                            print('Cant do %s' %name)
                            continue
                        type_dict[arche][name] = param['oneOrMore']['element']['data']['@type']
            else:
                param = schema_dict['interleave'][op_el]
                name = param['@name']
                if 'interleave' in param:
                        print('%s Cant do man' %param)
                        continue
                if 'data' in param:
                    param_dict[arche]['one'].append(name)
                    type_dict[arche][name] = param['data']['@type']
                    continue
                if 'oneOrMore' in param:
                    param_dict[arche]['oneormore'].append(name)
                    tag_dict[arche][name] = param['oneOrMore']['element']['@name']
                    if 'interleave' in param['oneOrMore']['element'].keys():
                        print('Cant do %s' %param)
                        continue
                    type_dict[arche][name] = param['oneOrMore']['element']['data']['@type']
        else:
            optional = True
            if isinstance(schema_dict['interleave'][op_el], list):
                for param in schema_dict['interleave'][op_el]:
                    param = param['element']
                    name = param['@name']
                    optional_dict[arche].append(name)
                    if 'interleave' in param:
                        print('%s Cant do man' %param)
                        continue
                    if 'data' in param:
                        param_dict[arche]['one'].append(name)
                        type_dict[arche][name] = param['data']['@type']
                        continue
                    if 'oneOrMore' in param:
                        param_dict[arche]['oneormore'].append(name)
                        tag_dict[arche][name] = param['oneOrMore']['element']['@name']
                        type_dict[arche][name] = param['oneOrMore']['element']['data']['@type']        
            else:
                param = schema_dict['interleave'][op_el]['element']
                name = param['@name']
                optional_dict[arche].append(param['@name'])
                if 'interleave' in param:
                        print('%s Cant do man' %param)
                        continue
                if 'data' in param:
                    param_dict[arche]['one'].append(name)
                    type_dict[arche][name] = param['data']['@type']
                    continue
                if 'oneOrMore' in param:
                    param_dict[arche]['oneormore'].append(name)
                    tag_dict[arche][name] = param['oneOrMore']['element']['@name']
                    type_dict[arche][name] = param['oneOrMore']['element']['data']['@type']   

            

Cant do in_streams
Cant do streams


In [193]:
print(type_dict[':cycamore:Reactor'])
print('\n\n')
print(tag_dict[':cycamore:Reactor'])
print('\n\n')
print(param_dict[':cycamore:Reactor'])
print('\n\n')
print(optional_dict[':cycamore:Reactor'])
print('\n\n')

{'fuel_incommods': 'string', 'fuel_inrecipes': 'string', 'fuel_outcommods': 'string', 'fuel_outrecipes': 'string', 'assem_size': 'double', 'n_assem_batch': 'int', 'fuel_prefs': 'double', 'recipe_change_times': 'int', 'recipe_change_commods': 'string', 'recipe_change_in': 'string', 'recipe_change_out': 'string', 'n_assem_core': 'int', 'n_assem_fresh': 'int', 'n_assem_spent': 'int', 'cycle_time': 'int', 'refuel_time': 'int', 'cycle_step': 'int', 'power_cap': 'double', 'power_name': 'string', 'side_products': 'string', 'side_product_quantity': 'double', 'pref_change_times': 'int', 'pref_change_commods': 'string', 'pref_change_values': 'double', 'latitude': 'double', 'longitude': 'double'}



{'fuel_incommods': 'val', 'fuel_inrecipes': 'val', 'fuel_outcommods': 'val', 'fuel_outrecipes': 'val', 'fuel_prefs': 'val', 'recipe_change_times': 'val', 'recipe_change_commods': 'val', 'recipe_change_in': 'val', 'recipe_change_out': 'val', 'side_products': 'val', 'side_product_quantity': 'val', 'pref

==================================================================================================================

In [130]:
for i in j['schema']:
    tag_dict = {i: {}}
    fac_dict = xmltodict.parse(j['schema'][i])['interleave']
    for el_op in fac_dict:
        if el_op == 'element':
            optional = False
        elif el_op == 'optional':
            optional = True
        for param in fac_dict[el_op]:
            param_dict = {}
            param_name = param['@name']
            if 'interleave' in param.keys():
                param_dict = interleave_digger(param, param_dict)
            if 'oneOrMore' in param.keys():
                param_dict = oneOrMore_digger(param, param_dict)
            if 'data' in param.keys():
                param_dict = data_digger(param, param_dict)
                
def interleave_digger(d, param_dict):
    for i in d['interleave']['element']:
        new_dict = d['element']
        name = new_dict['@name']
        if 'interleave' in new_dict.keys():
            interleave_digger(new_dict, param_dict)
        elif 'oneOrMore' in new_dict.keys():
            oneOrMore_digger(new_dict, param_dict)
        elif 'data' in new_dict.keys():
            data_digger(new_dict, param_dict)
    
def oneOrMore_digger(d):
    
    if 'interleave' in new_dict.keys():
        interleave_digger(new_dict, param_dict)
    elif 'oneOrMore' in new_dict.keys():
        oneOrMore_digger(new_dict, param_dict)
    elif 'data' in new_dict.keys():
        data_digger(new_dict, param_dict)

def data_digger(d):
    
    
        
"""        
        for param in j['schema'][i]['interleave'][el_op]:
            param_name = param['@name'] 
            if 'oneOrMore' in param.keys():
                oneormore = True
                tag = param_name['oneOrMore']['element']['@name']
                # if 'interleave' in param_name['oneOrMore']['element']
                    
            else:
                oneormore = False
"""         

:agents:KFacility


TypeError: string indices must be integers

In [47]:

doc = j['annotations'][':cycamore:Separations']['doc']
variables = j['annotations'][':cycamore:Separations']['vars']
def parse_variable_dict(variable_dict):
    for name, specs in variable_dict.items():
        if isinstance(specs, str):
            print('WHY IS %s a string\n' %specs)
            continue 
    
        if 'doc' in specs.keys():
            doc_ = specs['doc']
        elif 'tooltip' in specs.keys():
            doc_ = specs['tooltip']
        else:
            doc_ = ''
        
        if 'type' in specs.keys():
            type_ = specs['type']
        else:
            type_ = ''
            
        if 'default' in specs.keys():
            default_ = specs['default']
        else:
            default_ = ''
        
        print('%s (%s):\n%s\n%s' %(name, type_, doc_, 'Default:%s'%default_))
        print('\n')
    
parse_variable_dict(variables)
        

feed (['cyclus::toolkit::ResBuf', 'cyclus::Material']):

Default:


feed_commod_prefs (['std::vector', 'double']):
Feed commodity request preferences for each of the given feed commodities (same order). If unspecified, default is to use 1.0 for all preferences.
Default:[]


feed_commods (['std::vector', 'std::string']):
Ordered list of commodities on which to request feed material to separate. Order only matters for matching up with feed commodity preferences if specified.
Default:


feed_recipe (std::string):
Name for recipe to be used in feed requests. Empty string results in use of a dummy recipe.
Default:


feedbuf_size (double):
Maximum amount of feed material to keep on hand.
Default:


latitude (double):
Latitude of the agent's geographical position. The value should be expressed in degrees as a double.
Default:0.0


leftover (['cyclus::toolkit::ResBuf', 'cyclus::Material']):

Default:


leftover_commod (std::string):
Commodity on which to trade the leftover separated material s

In [23]:
for key, val in variables.items():
    print(key)
    try:
        print(val['shape'])
    except:
        print(val)

feed
[-1, -1]
feed_commod_prefs
[-1, -1]
feed_commods
[-1, -1]
feed_recipe
[-1]
feedbuf_size
[-1]
latitude
[-1]
leftover
[-1, -1]
leftover_commod
[-1]
leftoverbuf_size
[-1]
longitude
[-1]
streams
streams_
streams_
[-1, -1, -1, -1, -1, -1, -1]
throughput
[-1]


In [8]:
import xmltodict

In [24]:
proto_dict = {}
with open('./output_c98/prototypes.xml', 'r') as f:
    xml_list = xmltodict.parse(f.read())['root']['facility']
    for facility in xml_list:
        print(facility)
        print('\n\n')
        facility_name = facility['name']
        archetype = list(facility['config'].keys())[0]
        # proto_dict[facility_name] = {'archetype': archetype,
        #                              'config': facility['config'][archetype]}

OrderedDict([('name', 'enrichment'), ('config', OrderedDict([('Enrichment', OrderedDict([('feed_commod', 'natl_u'), ('feed_recipe', 'natl_u'), ('product_commod', 'uox'), ('tails_assay', '0.003'), ('tails_commod', 'waste'), ('swu_capacity', '1e100'), ('initial_feed', '1e100')]))]))])



OrderedDict([('name', 'separations'), ('config', OrderedDict([('Separations', OrderedDict([('streams', OrderedDict([('item', OrderedDict([('commod', 'sep_stream'), ('info', OrderedDict([('buf_size', '1e100'), ('efficiencies', OrderedDict([('item', OrderedDict([('comp', 'Pu'), ('eff', '.99')]))]))]))]))])), ('leftover_commod', 'waste'), ('throughput', '30001'), ('feedbuf_size', '30001'), ('feed_commods', OrderedDict([('val', 'spent_uox')])), ('feed_commod_prefs', OrderedDict([('val', '2.0')]))]))]))])



OrderedDict([('name', 'fuelfab'), ('config', OrderedDict([('FuelFab', OrderedDict([('fill_commods', OrderedDict([('val', 'depleted_u')])), ('fill_recipe', 'depleted_u'), ('fill_size', '30001'), ('fiss_com

In [30]:
from collections import OrderedDict
proto_dict = {'enrichment': {'archetype': 'Enrichment', 'config': OrderedDict([('feed_commod', 'natl_u'), ('feed_recipe', 'natl_u'), ('product_commod', 'uox'), ('tails_assay', '0.003'), ('tails_commod', 'waste'), ('swu_capacity', '1e100'), ('initial_feed', '1e100')])}, 'separations': {'archetype': 'Separations', 'config': OrderedDict([('streams', OrderedDict([('item', OrderedDict([('commod', 'sep_stream'), ('info', OrderedDict([('buf_size', '1e100'), ('efficiencies', OrderedDict([('item', OrderedDict([('comp', 'Pu'), ('eff', '.99')]))]))]))]))])), ('leftover_commod', 'waste'), ('throughput', '30001'), ('feedbuf_size', '30001'), ('feed_commods', OrderedDict([('val', 'spent_uox')])), ('feed_commod_prefs', OrderedDict([('val', '2.0')]))])}, 'fuelfab': {'archetype': 'FuelFab', 'config': OrderedDict([('fill_commods', OrderedDict([('val', 'depleted_u')])), ('fill_recipe', 'depleted_u'), ('fill_size', '30001'), ('fiss_commods', OrderedDict([('val', 'sep_stream')])), ('fiss_size', '15000'), ('spectrum', 'thermal'), ('outcommod', 'mox'), ('throughput', '30001')])}, 'reactor': {'archetype': 'Reactor', 'config': OrderedDict([('fuel_inrecipes', OrderedDict([('val', ['fresh_uox', 'fresh_mox'])])), ('fuel_outrecipes', OrderedDict([('val', ['spent_uox', 'spent_mox'])])), ('fuel_incommods', OrderedDict([('val', ['uox', 'mox'])])), ('fuel_outcommods', OrderedDict([('val', ['spent_uox', 'waste'])])), ('fuel_prefs', OrderedDict([('val', ['1.0', '2.0'])])), ('cycle_time', '17'), ('refuel_time', '2'), ('assem_size', '30000'), ('n_assem_core', '3'), ('n_assem_batch', '1')])}, 'repo': {'archetype': 'Sink', 'config': OrderedDict([('in_commods', OrderedDict([('val', 'waste')])), ('capacity', '1e100')])}, 'depleted_src': {'archetype': 'Source', 'config': OrderedDict([('outcommod', 'depleted_u'), ('outrecipe', 'depleted_u')])}}
proto_dict

{'enrichment': {'archetype': 'Enrichment',
  'config': OrderedDict([('feed_commod', 'natl_u'),
               ('feed_recipe', 'natl_u'),
               ('product_commod', 'uox'),
               ('tails_assay', '0.003'),
               ('tails_commod', 'waste'),
               ('swu_capacity', '1e100'),
               ('initial_feed', '1e100')])},
 'separations': {'archetype': 'Separations',
  'config': OrderedDict([('streams',
                OrderedDict([('item',
                              OrderedDict([('commod', 'sep_stream'),
                                           ('info',
                                            OrderedDict([('buf_size', '1e100'),
                                                         ('efficiencies',
                                                          OrderedDict([('item',
                                                                        OrderedDict([('comp',
                                                                                  

In [43]:
new_dict = {'root': {'facility': []}}
proto_dict = {'reac1': {'archetype': 'Reactor', 'config': {'Reactor': {'fuel_incommods': {'val': ['inc1', 'inc2']}, 'assem_size': '30'}}}}
for name, config in proto_dict.items():
    facility_dict = {}
    facility_dict['name'] = name
    print(config['config'])
    facility_dict['config'] = {config['archetype']: config['config']}
    new_dict['root']['facility'].append(facility_dict)

f = open('./test.xml', 'w')
f.write(xmltodict.unparse(new_dict, pretty=True))
f.close()

{'Reactor': {'fuel_incommods': {'val': ['inc1', 'inc2']}, 'assem_size': '30'}}


In [25]:
x = OrderedDict([('name', 'depleted_src'), ('config', OrderedDict([('Source', OrderedDict([('outcommod', 'depleted_u'), ('outrecipe', 'depleted_u')]))]))])
dict(x)

{'name': 'depleted_src',
 'config': OrderedDict([('Source',
               OrderedDict([('outcommod', 'depleted_u'),
                            ('outrecipe', 'depleted_u')]))])}

In [138]:
x ="""<facility>
    <name>mox_reprocessing</name>
    <config>
      <Separations>
         <feed_commods>   <val>cool_mox_waste</val>  </feed_commods>
         <feed_commod_prefs> <val>2.0</val> </feed_commod_prefs>
         <feedbuf_size>1E100</feedbuf_size>
         <throughput>1E100</throughput>
         <leftover_commod>mox_reprocess_waste</leftover_commod>
         <leftoverbuf_size>1E100</leftoverbuf_size>
         <streams>
          <item>
            <commod>mox_TRU</commod>
            <info>
              <buf_size>1E100</buf_size>
              <efficiencies>
                <item>
                  <comp>Np</comp> <eff>.998</eff>
                </item>
                <item>
                  <comp>Am</comp> <eff>.998</eff>
                </item>
                <item>
                  <comp>Cm</comp> <eff>.998</eff>
                </item>
                <item>
                  <comp>Pu</comp> <eff>.998</eff>
                </item>
              </efficiencies>
            </info>
          </item>
          <item>
            <commod>mox_U</commod>
            <info>
              <buf_size>1E100</buf_size>
              <efficiencies>
                <item>
                  <comp>U</comp> <eff>.998</eff>
                </item>
              </efficiencies>
            </info>
          </item>
        </streams>
      </Separations>
    </config>
</facility>"""
xmltodict.parse(x)['facility']['config']['Separations']['streams']

OrderedDict([('item',
              [OrderedDict([('commod', 'mox_TRU'),
                            ('info',
                             OrderedDict([('buf_size', '1E100'),
                                          ('efficiencies',
                                           OrderedDict([('item',
                                                         [OrderedDict([('comp',
                                                                        'Np'),
                                                                       ('eff',
                                                                        '.998')]),
                                                          OrderedDict([('comp',
                                                                        'Am'),
                                                                       ('eff',
                                                                        '.998')]),
                                                          O